# Gold Layer - Fato Vendas

In [0]:
%run ./00_Setup_Environment

In [0]:
df_header = spark.table("silver.sales_orders")


df_dim_date = spark.table("gold.dim_date")
df_dim_customer = spark.table("gold.dim_customer")
df_dim_address = spark.table("gold.dim_address")

In [0]:
df_fact_sales = df_header.alias("s") \
    .join(df_dim_date.alias("d"), 
          col("s.order_date") == col("d.date"), 
          "left") \
    .join(df_dim_customer.alias("c"), 
          col("s.customer_id") == col("c.customer_id"), 
          "left") \
    .join(df_dim_address.alias("a_ship"), 
          col("s.ship_to_address_id") == col("a_ship.address_id"), 
          "left") \
    .join(df_dim_address.alias("a_bill"), 
          col("s.bill_to_address_id") == col("a_bill.address_id"), 
          "left") \
    .select(
        # Surrogate Keys
        col("d.date_key").alias("order_date_key"),
        col("c.customer_key"),
        col("a_ship.address_key").alias("ship_address_key"),
        col("a_bill.address_key").alias("bill_address_key"),
        
        # Business Keys
        col("s.order_id"),
        col("s.purchase_order_number"),
        
        #
        col("s.order_subtotal"),
        col("s.order_tax"),
        col("s.order_freight"),
        col("s.order_total"),
        
        # Métricas de negócio
        col("s.is_online_order"),
        col("s.order_status"),
        
        # Metadados
        col("s.processed_timestamp")
    )


In [0]:
path = f"{gold_path}/fact_sales"
df_fact_sales.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save(path)

df_fact_sales.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold.fact_sales")

count = df_fact_sales.count()
log_etl("fact_sales", "gold", "SUCCESS", count)

print(f"gold.fact_sales criada com {count} registros")